In [5]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.models import logcnl_avail
from biogeme.expressions import Beta, DefineVariable, log
from biogeme.expressions import Beta, DefineVariable, log

pandas = pd.read_table("telephone.dat")
database = db.Database("telephone",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_BM	 = Beta('ASC_BM',0,None,None,0)
ASC_EF	 = Beta('ASC_EF',0,None,None,0)
ASC_LF	 = Beta('ASC_LF',0,None,None,0)
ASC_MF	 = Beta('ASC_MF',0,None,None,0)
ASC_SM	 = Beta('ASC_SM',0,None,None,1)
B_COST	 = Beta('B_COST',0,None,None,0)
#
# parameters relevant to the nests
N_FLAT = Beta('N_FLAT',2.16,None,None,1)
N_MEAS = Beta('N_MEAS',2.16,None,None,1)
#
a_FLAT_LF = Beta('a_FLAT_LF',0.5,0,1,1)
a_MEAS_LF = Beta('a_MEAS_LF',0.5,0,1,1)
a_FLAT_EF = Beta('a_FLAT_EF',1,0,1,1)
a_FLAT_MF = Beta('a_FLAT_MF',1,0,1,1)
a_MEAS_BM = Beta('a_MEAS_BM',1,0,1,1)
a_MEAS_SM = Beta('a_MEAS_SM',1,0,1,1)

# Define here arithmetic expressions for names that are not directly
# available from the data

logcostBM  = DefineVariable('logcostBM', log(cost1),database)
logcostSM  = DefineVariable('logcostSM', log(cost2),database)
logcostLF  = DefineVariable('logcostLF', log(cost3),database)
logcostEF  = DefineVariable('logcostEF', log(cost4),database)
logcostMF  = DefineVariable('logcostMF', log(cost5),database)

#Utilities
V_BM = ASC_BM + B_COST * logcostBM
V_SM = ASC_SM + B_COST * logcostSM
V_LF = ASC_LF + B_COST * logcostLF
V_EF = ASC_EF + B_COST * logcostEF
V_MF = ASC_MF + B_COST * logcostMF

V = {1: V_BM, 2: V_SM, 3: V_LF, 4: V_EF, 5: V_MF}
avail = {1: avail1, 2: avail2, 3: avail3, 4: avail4, 5: avail5}

#Definitions of nests
alpha_N_FLAT = {1: 0, 2: 0, 3: a_FLAT_LF, 4: a_FLAT_EF, 5: a_FLAT_MF}
alpha_N_MEAS = {1: a_MEAS_BM, 2: a_MEAS_SM, 3: a_MEAS_LF, 4: 0, 5: 0}

nest_N_FLAT = N_FLAT, alpha_N_FLAT
nest_N_MEAS = N_MEAS, alpha_N_MEAS

nests = nest_N_FLAT, nest_N_MEAS

# CNL with fixed alphas
logprob = logcnl_avail(V, avail, nests, choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "GEV_Tel_CNL_fix"
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
display(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")



,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_BM,-0.356,0.0735,-4.85,1.27e-06,0.073,-4.87,1.09e-06
ASC_EF,0.465,0.419,1.11,0.267,0.395,1.18,0.24
ASC_LF,0.867,0.085,10.2,0,0.087,9.96,0
ASC_MF,0.791,0.165,4.8,1.59e-06,0.161,4.9,9.43e-07
B_COST,-1.24,0.129,-9.65,0,0.132,-9.41,0


Nbr of observations: 434
LL(0) =    -612.574
LL(beta) = -480.146
rho bar square = 0.208
Output file: GEV_Tel_CNL_fix~01.html


In [2]:
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 2)
pandas

,choice,area,users,inc,age0,age1,age2,age3,age4,age5,age6,age7,status,employ,cost1,cost2,cost3,cost4,cost5,avail1,avail2,avail3,avail4,avail5,ones,logcostBM,logcostSM,logcostLF,logcostEF,logcostMF
0,2,1,2,4,0,0,0,1,1,0,0,0,2,2,5.82,5.78,12.8,1e+06,23.3,1,1,1,0,1,1,1.76,1.75,2.55,13.8,3.15
1,3,1,1,2,0,0,0,0,0,0,0,1,5,1,3.52,5.78,12.3,1e+06,23.3,1,1,1,0,1,1,1.26,1.75,2.51,13.8,3.15
2,1,2,2,4,0,0,0,2,0,0,0,0,2,2,5.09,5.78,11.5,1e+06,28.3,1,1,1,0,1,1,1.63,1.75,2.44,13.8,3.34
3,3,2,2,3,1,0,0,0,2,0,0,0,2,1,4.75,5.78,10.5,1e+06,28.3,1,1,1,0,1,1,1.56,1.75,2.35,13.8,3.34
4,3,2,6,3,1,3,2,1,0,2,0,0,2,2,8.55,7.05,14.3,1e+06,28.3,1,1,1,0,1,1,2.15,1.95,2.66,13.8,3.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,2,5,2,3,2,0,0,1,1,0,0,0,2,2,10.3,8.78,8.48,1e+06,1e+06,1,1,1,0,0,1,2.33,2.17,2.14,13.8,13.8
430,3,5,2,3,0,0,0,1,1,0,0,0,2,2,8.56,7.06,8.48,1e+06,1e+06,1,1,1,0,0,1,2.15,1.95,2.14,13.8,13.8
431,2,5,3,4,0,0,0,1,0,1,1,0,2,3,7.19,5.78,8.48,1e+06,1e+06,1,1,1,0,0,1,1.97,1.75,2.14,13.8,13.8
432,2,5,1,2,0,0,0,0,0,0,0,1,3,0,5.96,5.78,8.48,1e+06,1e+06,1,1,1,0,0,1,1.79,1.75,2.14,13.8,13.8


In [3]:
pandas.describe()

,choice,area,users,inc,age0,age1,age2,age3,age4,age5,age6,age7,status,employ,cost1,cost2,cost3,cost4,cost5,avail1,avail2,avail3,avail4,avail5,ones,logcostBM,logcostSM,logcostLF,logcostEF,logcostMF
count,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434
mean,2.65,2.93,2.3,2.53,0.21,0.226,0.242,0.406,0.435,0.362,0.311,0.38,2.22,1.07,11.7,11.5,14.8,9.7e+05,3.55e+05,1,1,1,0.03,0.645,1,2.08,2.12,2.5,13.5,7.03
std,1.17,1.65,1.28,1.28,0.526,0.576,0.669,0.711,0.727,0.67,0.614,0.652,0.91,0.886,24.1,23.9,23.6,1.71e+05,4.79e+05,0,0,0,0.171,0.479,0,0.702,0.598,0.455,1.82,5.04
min,1,1,1,1,0,0,0,0,0,0,0,0,1,0,3.28,5.78,7.03,10.5,23.3,1,1,1,0,0,1,1.19,1.75,1.95,2.35,3.15
25%,2,2,1,1,0,0,0,0,0,0,0,0,2,0,4.82,5.78,10.2,1e+06,28.3,1,1,1,0,0,1,1.57,1.75,2.32,13.8,3.34
50%,3,2,2,2,0,0,0,0,0,0,0,0,2,1,6.95,5.78,10.9,1e+06,28.3,1,1,1,0,1,1,1.94,1.75,2.39,13.8,3.34
75%,3,5,3,3,0,0,0,1,1,1,0,1,3,2,11.7,10.2,13.1,1e+06,1e+06,1,1,1,0,1,1,2.46,2.32,2.57,13.8,13.8
max,5,5,6,5,4,3,4,3,2,2,3,2,5,3,434,433,436,1e+06,1e+06,1,1,1,1,1,1,6.07,6.07,6.08,13.8,13.8
